In [ ]:
# Author: Sachin K Salim
# Date:    May 2021
# 
# Description: Implementation of Gibbs Sampling from the paper "GIBBS 
#               SAMPLING FOR THE UNINITIATED" by Philip Resnik, Eric Hardisty

In [ ]:
"""
Notations
---------
V : number of words in the vocabulary
i : index of the word in the vocabulary

N : number of docs in the corpus
j : index for documents

Z : number of labels (=2)
x : index of label


γ_π : 1-D ndarray of len Z
    hyperparameters of the Beta distribution
γ_θ : 1-D ndarray of length V
        hyperparameter vector for the multinomial prior
        

labels : 1-D ndarray of len N

class_count : 1-D ndarray of length Z
    Number of docs labeled x
        
docs : 2-D ndarray of size NxV
    List of all docs in BoW format
docs_0, docs_1 : 2-D ndarrays of size class_count_x*V
    List of docs labeled x

word_count : 2-D ndarray of size ZxV
    number of times word i occurs in docs labeled x
    
θ : 2-D ndarray of size ZxV
    θ_xi stands for probability of word i from distribution of class x
θ_r : 1-D ndarray of length V
    θ_r = (θ[0]/θ[1])
"""

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Source code of load_data(): https://medium.com/swlh/sentiment-analysis-with-bag-of-words-4b9786e967ca
# Dataset: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# For cleaning the data
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# For creating a bag of words
from sklearn.feature_extraction.text import CountVectorizer

def load_data(doc_count = None, vocabulary_size = 1500):
    """loads data to do sentiment analysis

    Parameters
    ----------
    doc_count : int (optional - Default(None))
        Number of docs to be loaded from dataset
    vocabulary_size : int (optional - Default(1500))
        The number of words in vocabulary

    Returns
    -------
    x : docs
    y : labels
    """
    
    # Loading the data
    dataset = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv", header = None, nrows = doc_count)
    dataset = dataset.rename(columns={0: "Review", 1: "Liked"})
    dataset['bLiked'] = (dataset['Liked'] == "positive").astype(int)
    N = np.size(dataset, axis=0)
    
    # Cleaning the data
    corpus = []
    ps = PorterStemmer()
    for i in range(0, N):
        review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
        review = review.lower()
        review = review.split()
        all_stopwords = stopwords.words('english')
        all_stopwords.remove('not')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
        
    # Creating a bag of words
    cv = CountVectorizer(max_features = vocabulary_size)
    x = cv.fit_transform(corpus).toarray()
    y = dataset['bLiked'].values
    
    return x, y

In [ ]:
def initialize_training_indices(training_ratio = 0.8):
    """Randomly initializes training indices using training_ratio

    Parameters
    ----------
    training_ratio : float (optional - Default(0.8))
        The approx ratio of training data to total data

    Returns
    -------
    train_indices : 1-D ndarray of len N
        a boolean array with True at indices of training data
    """
    tdr = training_ratio
    train_indices = np.random.choice(a=[True,False], size=N, p=[tdr, 1-tdr])
    return train_indices

def is_training_doc(train_indices, j):
    """Determines if doc_j is a training data"""
    return train_indices[j]

In [ ]:
# Initialization of the labels
def __initialize_labels(N, Z):
    """Initialization of the labels

    Parameters
    ----------
    N : number of docs in the corpus
    Z : number of labels

    Returns
    -------
    γ_π : 1-D ndarray of len Z
    labels : 1-D ndarray of len N
    """
    
    γ_π = np.ones(Z, dtype=int) # (γ_π_0, γ_π_1)

    π = np.random.beta(γ_π[0], γ_π[1])
    labels = np.random.binomial(n=1, p=π, size=N)
    # assigning observed labels
    labels[train_indices] = labels_orig[train_indices]
    
    return γ_π, labels

# Initialization of θ
def __initialize_θ(V, Z):
    """Initialization of θ

    Parameters
    ----------
    V : number of words in the vocabulary
    Z : number of labels

    Returns
    -------
    γ_θ : 1-D ndarray of length V
    θ_r : 1-D ndarray of length V
    """
    
    # hyperparameter vector for the multinomial prior
    γ_θ = np.ones(V, dtype=int)

    θ = np.random.dirichlet(γ_θ, size=2)
    θ_r = θ[0]/θ[1]
    
    return γ_θ, θ_r

def initialize(V, N, Z):
    """Initialization of the Sampler

    Parameters
    ----------
    V : number of words in the vocabulary
    N : number of docs in the corpus
    Z : number of labels

    Returns
    -------
    γ_π : 1-D ndarray of len Z
    labels : 1-D ndarray of len N
    γ_θ : 1-D ndarray of length V
    θ_r : 1-D ndarray of length V
    """
    
    γ_π, labels = __initialize_labels(N, Z)
    γ_θ, θ_r = __initialize_θ(V, Z)
    
    return γ_π, labels, γ_θ, θ_r

In [ ]:
def update_label(doc_j, class_count, θ_r, γ_π):
    """Updating label_j by sampling from the conditional distribution

    Parameters
    ----------
    doc_j (const) : 1-D ndarray of len V
    class_count : 1-D ndarray of length Z
    θ_r : 1-D ndarray of length V
    γ_π (const) :  1-D ndarray of len Z

    Returns
    -------
    L_j : int ∈ {0, 1}
        updated label at index j
    """
    
    # In section 2.5.1, paper asks us to compute value_0 and value_1 separately and then
    # normalize it to find π = value_0/(value_0+value_1). But value_0, value_1 may become
    # infinitesimally small and thus program might treat it as 0. To avoid this, we instead
    # compute f=value_0/value_1 instead of computing the values separately
    term_1 = (class_count[0] + γ_π[0] - 1)/(class_count[1] + γ_π[1] - 1)
    term_2 = np.product(θ_r**doc_j)
    f = term_1 * term_2
    
    # π: probability of label 0
    π = 1/(1 + f)
    
    L_j = np.random.binomial(n=1, p=π)
    return L_j

In [ ]:
def __update_class_counts(labels):
    """
    Updates class-counts which are used to resample
    labels at the beginning of every iteration.
    
    Parameters
    ----------
    labels : 1-D ndarray of length N
    
    Returns
    -------
    class_count : 1-D ndarray of length Z
    """
    
    # class_count_x: No. of docs labeled x
    class_count_0 = np.sum(labels == 0)
    N = labels.size
    class_count_1 = N - class_count_0
    class_count = np.array([class_count_0, class_count_1])
    
    return class_count

def __update_word_counts(docs):
    """
    Updates word-counts which are used to resample
    theta at the beginning of every iteration.
    
    Parameters
    ----------
    docs (const) : 2-D ndarray of size NxV
    
    Returns
    -------
    word_count : 2-D ndarray of size ZxV
    """
    
    def __split(arr, cond):
        """Util function to split an array based on a condition"""
        return (arr[cond], arr[~cond])

    # docs_x: The set of documents labeled x
    docs_0, docs_1 = __split(docs, labels == 0)
    
    # word_count_x: No. of times word i occurs in docs__x
    word_count = np.array([np.sum(docs_0, axis=0), np.sum(docs_1, axis=0)])
    
    return word_count
    

def update_counts(labels, docs):
    """
    Updates class-counts and word-counts which are used to resample
    labels and theta respectively, at the beginning of every iteration.
    
    Parameters
    ----------
    docs (const) : 2-D ndarray of size NxV
    labels : 1-D ndarray of length N
    
    Returns
    -------
    class_count : 1-D ndarray of length Z
    word_count : 2-D ndarray of size ZxV
    """
    
    class_count = __update_class_counts(labels)
    word_count = __update_word_counts(docs)
    
    return class_count, word_count

def update_word_count_x(word_count_x, doc, change):
    """
    Updates word-count-x (labeled x) by either reducing
    or incrementing the doc's word-counts
    
    Parameters
    ----------
    word_count_x
    doc (const) : 1-D ndarray of length V
    change : either +1 or -1
    
    """
    word_count_x += change * doc
    return word_count_x

In [ ]:
def update_theta(word_count, γ_θ):
    """
    Parameters
    ----------
    word_count : 2-D ndarray of size ZxV
    γ_θ : 2-D ndarray of size ZxV
    
    Returns
    -------
    θ_r : 1-D ndarray of length V
    """    
    θ_0 = np.random.dirichlet(word_count[0] + γ_θ)
    θ_1 = np.random.dirichlet(word_count[1] + γ_θ)
    θ_r = θ_0/θ_1
    return θ_r

In [ ]:
def calculate_error(labels_sum, labels_orig, no_of_iterations, test_indices):
    labels_exp = labels_sum/no_of_iterations
    labels_exp = labels_exp.round().astype(int)
    labels_error = labels_exp[test_indices]-labels_orig[test_indices]
    return labels_error

def calculate_accuracy(labels_error):
    accuracy = 100*(1 - np.mean(np.square(labels_error)))
    return accuracy

In [ ]:
docs, labels_orig = load_data()

Z = 2 # Number of labels
N = np.size(docs, axis=0) # Number of documents
V = np.size(docs, axis=1) # Number of words in the vocabulary
V, N, Z

In [ ]:
# Re-run from here ->
train_indices = initialize_training_indices(training_ratio = 0)

γ_π, labels, γ_θ, θ_r = initialize(V, N, Z)

In [ ]:
T = 20 # number of iterations
B = 0 # burn-in

labels_sum = np.zeros(N, dtype=int)

for t in np.arange(1, T+1):
    class_count, word_count = update_counts(labels, docs)
    
    for j in np.arange(N):
         # if doc_j is a training doc, skip the iteration
        if(is_training_doc(train_indices, j)):
            continue

        x = labels[j]
        doc = docs[j]
        
        # For each word in doc_j, subtract its count in doc_j from its corresponding label's word-count
        word_count[x] = update_word_count_x(word_count[x], doc, -1)
        
        # Subtract 1 from corresponding label's document-count
        class_count[x] -= 1
        
        # Update L_j
        x = labels[j] = update_label(doc, class_count, θ_r, γ_π)
        
        # For each word in doc_j, add its count in doc_j to its corresponding label's word-count
        word_count[x] = update_word_count_x(word_count[x], doc, +1)
        
        # Add 1 to corresponding label's document-count
        class_count[x] += 1
        
    # θ: theta
    θ_r = update_theta(word_count, γ_θ)
    
    if(t<=B):
        continue
    
    labels_sum += labels

labels_error = calculate_error(labels_sum, labels_orig, T-B, ~train_indices)
accuracy = calculate_accuracy(labels_error)